# My own sql database

In [7]:
import sqlite3

# Connect to the database (creates a new database if it doesn't exist)
conn = sqlite3.connect('gibberish.sqlite')

# Create a cursor object to execute SQL statements
cursor = conn.cursor()

# Create the first table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS y (
        id INTEGER PRIMARY KEY,
        N TEXT,
        H REAL
    )
''')
y_data = [
    (1, 'Alice', 205.1),
    (2, 'Bob', 201.2),
    (3, 'Cat', 199.4)
]
cursor.executemany(
    'INSERT INTO y VALUES (?, ?, ?)', 
    y_data
)


# Create the second table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS gmass (
        id INTEGER PRIMARY KEY,
        N TEXT,
        W REAL
    )
''')
gmass_data = [
    (1, 'Alice',175.2 ),
    (2, 'Bob', 156.2),
    (3, 'Cat', 123.8)
]
cursor.executemany(
    'INSERT INTO gmass VALUES (?, ?, ?)', 
    gmass_data
)

# Commit the changes and close the connection
conn.commit()
conn.close()

# Queries

In [1]:
import sqlite3
conn = sqlite3.connect("gibberish.sqlite")

# Define a finction to be made into a tool. 
def run_sqlite_query(query):
    c =  conn.cursor() #Cursor is the object that allows us to access to the db.
    try:
        c.execute(query)
        return c.fetchall() 
    except sqlite3.OperationalError as err:
        return f"The following error occuerd: {str(err)}"

# check that query works
print(run_sqlite_query("""
SELECT * FROM gmass 
"""))
print(run_sqlite_query("""
SELECT * FROM y 
"""))

[(1, 'Alice', 175.2), (2, 'Bob', 156.2), (3, 'Cat', 123.8)]
[(1, 'Alice', 205.1), (2, 'Bob', 201.2), (3, 'Cat', 199.4)]


## Create query tool 

In [2]:
from langchain.tools import Tool

run_query_tool = Tool.from_function(
    name="run_sqlite_query", 
    description="Run a sqlite query.",
    func=run_sqlite_query,
)
# Check that it works:
run_query_tool("""
SELECT * FROM y 
""")

[(1, 'Alice', 205.1), (2, 'Bob', 201.2), (3, 'Cat', 199.4)]

## Agent aware of Query Tool

In [3]:
from langchain.agents import OpenAIFunctionsAgent

from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv() # for David Cherney's OpenAI account.
chat=ChatOpenAI()
tools=[run_query_tool]

prompt = ChatPromptTemplate(
    messages=[
        SystemMessage(
            content = (
                "You are an AI that has access to a sqlite database. "
                )
            ),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(
            variable_name = "agent_scratchpad"
            )
        ]
    )

agent = OpenAIFunctionsAgent(
    llm=chat,
    prompt=prompt,
    tools=tools
    # memory = 
)

agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    tools=tools # Must be the same as in agent.
)

nl_query="What is the average height of the people in the database?"
agent_executor(nl_query)

/opt/anaconda3/envs/langchain_course_env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.openai_functions_agent.base.OpenAIFunctionsAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use create_openai_functions_agent instead.
  warn_deprecated(
/opt/anaconda3/envs/langchain_course_env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `run_sqlite_query` with `SELECT AVG(height) FROM people`


The following error occuerd: no such table: peopleI apologize, but it seems that there is no "people" table in the database. Please make sure that the table name is correct.

> Finished chain.


{'input': 'What is the average height of the people in the database?',
 'output': 'I apologize, but it seems that there is no "people" table in the database. Please make sure that the table name is correct.'}

The LLM guessed that there was a table named people and a feature in that tables called height. It needs to be informed of the names of the taabls and the schema. To do that we will
1. create a function to list the tables and add that list to the systemprompt
2. create a function that returns the schema, create a tool for that function, and thus allow the agent to look up the schema as needed. 

# List tables function
This is needed so that the query tool can be informed of the names of the tables.

In [4]:
import sqlite3
conn = sqlite3.connect("gibberish.sqlite")

def list_tables():
    c = conn.cursor()
    # List tables in the database using SQL.
    c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    rows = c.fetchall() 
    # SQL format issue:
    # Each item in rows was a 2-tuple with first item a table name as a string, 
    # second item blank. Replace all toples with their fist entry.
    rows = [row[0] for row in rows if row[0] is not None]
    # Reformat as a string with the name of one table on each line. 
    tables = " ".join(rows)
    return tables

tables = list_tables()
tables

'y gmass'

## Tables informed Agent with Query Tool

In [5]:
from langchain.agents import OpenAIFunctionsAgent

from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv() # for David Cherney's OpenAI account.
chat=ChatOpenAI()
tools=[run_query_tool]

prompt = ChatPromptTemplate(
    messages=[
        SystemMessage(
            content = (
                "You are an AI that has access to a sqlite database. "
                f"The database has the following tables: {list_tables().split(' ')}\n"
                "Do not make any assumptions about what tables exist. "
                )
            ),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(
            variable_name = "agent_scratchpad"
            )
        ]
    )

agent = OpenAIFunctionsAgent(
    llm=chat,
    prompt=prompt,
    tools=tools
    # memory = 
)

agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    tools=tools # Must be the same as in agent.
)

nl_query="What is the average height of the people in the database?"
agent_executor(nl_query)



> Entering new AgentExecutor chain...

Invoking: `run_sqlite_query` with `SELECT AVG(height) FROM y`


The following error occuerd: no such column: height
Invoking: `run_sqlite_query` with `SELECT AVG(height) FROM gmass`


The following error occuerd: no such column: heightI'm sorry, but it seems that there is no column named "height" in any of the tables in the database. Please make sure that the column name is correct or provide more information about the structure of the tables.

> Finished chain.


{'input': 'What is the average height of the people in the database?',
 'output': 'I\'m sorry, but it seems that there is no column named "height" in any of the tables in the database. Please make sure that the column name is correct or provide more information about the structure of the tables.'}

The LLM 
- knew that the names of the tables were y and gmass
- assumed there was a feature called height in one of those tables and repeatedly went looking for it.

## Tool for examining schema

In [7]:
def describe_tables(table_names):
    c = conn.cursor()
    # Create a string that is a list of tables for SQL syntax. 
    # e.g. tables = "WHERE name in ('users','orders','products');" .
    tables = ', '.join("'"+ table + "'" for table in table_names) 
    schemas_tuples = c.execute(
        "SELECT sql "
        "FROM sqlite_master " 
        f"WHERE type='table' and name IN ({tables});"
        )
    schemas_tuples = list(schemas_tuples) # Seems to make it more permananet.
    schemas = [schema_tuple[0] for schema_tuple in schemas_tuples 
           if schema_tuple[0] is not None]
    schemas_string = '\n'.join(schemas)
    return schemas_string

describe_tables_tool = Tool.from_function(
    name="describe_tables",
    description=(
        "Gven a list of table names, " 
        + "returns the schema of those tables."
        ),
    func=describe_tables,
 #   args_schema=DescribeTablesArgsSchema 
)
print(describe_tables(table_names = list_tables().split(' ')))

CREATE TABLE y (
        id INTEGER PRIMARY KEY,
        N TEXT,
        H REAL
    )
CREATE TABLE gmass (
        id INTEGER PRIMARY KEY,
        N TEXT,
        W REAL
    )


## Table informed, query and schema tools

In [8]:
load_dotenv() # for David Cherney's OpenAI account.
chat=ChatOpenAI()
tools=[run_query_tool, describe_tables_tool]

prompt = ChatPromptTemplate(
    messages=[
        SystemMessage(
            content = (
                "You are an AI that has access to a sqlite database. "
                f"The database has the following tables: {list_tables().split(' ')}\n"
                "Do not make any assumptions about what tables or columns exist. "
                "Instead, pass the list of tables to the 'descibe_tables' function to obtain the schema of the tables."
                "You might need to join the tables on the feature N"
                )
            ),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(
            variable_name = "agent_scratchpad"
            )
        ]
    )

agent = OpenAIFunctionsAgent(
    llm=chat,
    prompt=prompt,
    tools=tools
    # memory = 
)

agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    tools=tools # Must be the same as in agent.
)

nl_query = """
What is the average ratio of height to weight of the people in the database?
"""
agent_executor(nl_query)



> Entering new AgentExecutor chain...

Invoking: `run_sqlite_query` with `{'query': 'SELECT AVG(height/weight) AS avg_ratio FROM people'}`


The following error occuerd: no such table: people
Invoking: `describe_tables` with `{'table_names': ['y', 'gmass']}`


CREATE TABLE y (
        id INTEGER PRIMARY KEY,
        N TEXT,
        H REAL
    )
CREATE TABLE gmass (
        id INTEGER PRIMARY KEY,
        N TEXT,
        W REAL
    )
Invoking: `run_sqlite_query` with `{'query': 'SELECT AVG(y.H/gmass.W) AS avg_ratio FROM y JOIN gmass ON y.N = gmass.N'}`


[(1.3564722162000782,)]The average ratio of height to weight of the people in the database is approximately 1.356.

> Finished chain.


{'input': '\nWhat is the average ratio of height to weight of the people in the database?\n',
 'output': 'The average ratio of height to weight of the people in the database is approximately 1.356.'}

----
**Take 1**:
```text
nl_query="What is the average height of the people in the database?"
agent_executor(nl_query)

> Entering new AgentExecutor chain...
Invoking: `run_sqlite_query` with `SELECT AVG(height) FROM people`
The following error occuerd: no such table: people
Invoking: `describe_tables` with `y`

CREATE TABLE y (
        id INTEGER PRIMARY KEY,
        N TEXT,
        H REAL
    )
Invoking: `run_sqlite_query` with `SELECT AVG(H) FROM y`


[(201.89999999999998,)]The average height of the people in the database is approximately 201.9.
 ```
So, the LLM guessed that H stood for height. 

----

Take 2 

```
nl_query = """
What is the average ratio of height to weight of the people in the database?
"""
agent_executor(nl_query)

Invoking: `run_sqlite_query` with `{'query': 'SELECT AVG(height / weight) as avg_ratio FROM y'}`


The following error occuerd: no such column: height
Invoking: `describe_tables` with `{'table_names': 'y'}`
...
Invoking: `describe_tables` with `{'table_names': 'gmass'}`


Invoking: `run_sqlite_query` with `{'query': 'SELECT AVG(H / W) as avg_ratio FROM y JOIN gmass ON y.N = gmass.N'}`


[(1.3564722162000782,)]

The average ratio of height to weight of the people in the database is approximately 1.36.
 ```

# Data dictionary DB

A data dictionary named `data_dictoinary.txt` will be fed through a text splitter on split charater "\n" and the result put in the chroma database `data_dictionary_vector_store`.

In [9]:
from langchain.document_loaders import TextLoader
from dotenv import load_dotenv 
from langchain.text_splitter import CharacterTextSplitter 
import tiktoken
# from langchain.embeddings import OpenAIEmbeddings # depricated
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings
    )
from langchain.vectorstores.chroma import Chroma
from langchain.text_splitter import CharacterTextSplitter 
from langchain_openai import OpenAIEmbeddings

In [44]:
embeddings = OpenAIEmbeddings()

text_splitter = CharacterTextSplitter(
    is_separator_regex=False, 
    chunk_size = 87, 
    separator = "\n", 
    chunk_overlap = 0  
)

loader = TextLoader("data_dictoinary.txt")
docs = loader.load_and_split(
    text_splitter=text_splitter,
)
db = Chroma.from_documents(
    documents = docs,
    embedding=embeddings,
    # Name the directory for the vector store.
    persist_directory="data_dictionary_vector_store2"
)
print(f"The databse is of type {type(db)} \n" 
      f"and has {db._collection.count()} entries.")

The databse is of type <class 'langchain_community.vectorstores.chroma.Chroma'> 
and has 6 entries.


In [14]:
embeddings = OpenAIEmbeddings()

db =  Chroma(
    persist_directory="data_dictionary_vector_store2", 
    embedding_function=embeddings)
# Chroma has complied and has a .as_retriever method on its database objects.


In [15]:
# test the vector store
results = db.similarity_search(
    query = "what symbol is used to represent a person's height inside a table?",
    k = 2 #Number of results to display. Default 4
    )
for result in results:
    print(result.page_content)

Column H : represents the height of the person in centimeters.
Table y: This table is named y because it contains data about peoples' height.


## Retrieval QA for natural language return

In [17]:
from langchain.chains import RetrievalQA 
# from langchain.chat_models import ChatOpenAI #depricated 
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma

"""For debugging, use these two lines. 
Warning: verbose is hard to read."""
# import langchain
# langchain.debug = True 

load_dotenv() 
chat = ChatOpenAI()
embeddings = OpenAIEmbeddings()
db =  Chroma(persist_directory="data_dictionary_vector_store2", embedding_function=embeddings)
# Chroma has complied and has a .as_retriever method on its database objects.
retriever = db.as_retriever()

chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=retriever,
    chain_type="stuff",
)

query = "What columns of what table represents the height of a person?"
result = chain(query)

print(result['result'])

The height of a person is represented by Column H in the table named y.


## Retrieval QA as a tool

In [24]:
def data_dictionary_qa(question):
    chat = ChatOpenAI()
    embeddings = OpenAIEmbeddings()
    db =  Chroma(
        persist_directory="data_dictionary_vector_store2", 
        embedding_function=embeddings
    )
    retriever = db.as_retriever()

    chain = RetrievalQA.from_chain_type(
        llm=chat,
        retriever=retriever,
        chain_type="stuff",
    )

    exchange = chain(question)
    return [exchange["result"]]

data_dictionary_qa("Which table has a feature about height and what is the name of that feature??")

['The table that has a feature about height is called "y". The name of the feature in table "y" that represents height is "H".']

In [25]:
data_dictionary_qa_tool = Tool.from_function(
    name="data_dictionary_qa", 
    description="""
    Takes in questions about the names and meaning of names of tables and columns 
    and gives answers based on the contents of the file data_dictionary.txt.
    """,
    func=data_dictionary_qa,
    return_direct=True
)
# Check that it works:
data_dictionary_qa_tool("""
which table has a feature about weight?""")

['The table "gmass" has a feature about weight.']

## Table informed, query, schema, and DD equipted

In [27]:
load_dotenv() # for David Cherney's OpenAI account.
chat=ChatOpenAI()
tools=[run_query_tool, describe_tables_tool, data_dictionary_qa_tool]

prompt = ChatPromptTemplate(messages=[
    SystemMessage(content=(
        "You are an AI that "
        "1. has access to an sqlite database "
        # "2. answers questions that involve data in that sqlite database."
        # "You can query that database with 'run_query_tool'."
        # "Do not make any assumptions about what tables exist in the database when making queries. "
        # "6 people will die if you make any assumptions about what features exist in tables when making queries."
        f"The database has the following tables: {list_tables().split(' ')}\n"
        # "To see the columns in a table you can view the schema of the table with 'descibe_tables'."
        # "To understand what the names of the tables and columns represent, "
        # "ask natural language question through data_dictionary_qa_tool"
    )),
    HumanMessagePromptTemplate.from_template(
        # Use the following steps to answer this question: 
        """
        {input} 
        """),
        # step 1. Pass the following to data_dictionary_qa_tool "Which tables and columns are relevant to this question: {input}" 
        # Step 2. Use the information from step 1 to the  run_query_tool
        # step 3: answer the question using the results of that query

    MessagesPlaceholder(variable_name = "agent_scratchpad")
    ]
)

agent = OpenAIFunctionsAgent(
    llm=chat,
    prompt=prompt,
    tools=tools
    # memory = 
)

agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    tools=tools # Must be the same as in agent.
)

nl_query = """
What is the average ratio of height to weight of the people in the database?
"""
agent_executor(nl_query)



> Entering new AgentExecutor chain...

Invoking: `run_sqlite_query` with `{'query': 'SELECT AVG(height/weight) as avg_ratio FROM people'}`


The following error occuerd: no such table: people
Invoking: `describe_tables` with `{'table_names': 'y'}`


CREATE TABLE y (
        id INTEGER PRIMARY KEY,
        N TEXT,
        H REAL
    )
Invoking: `run_sqlite_query` with `{'query': 'SELECT AVG(H) as avg_height FROM y'}`


[(201.89999999999998,)]
Invoking: `run_sqlite_query` with `{'query': 'SELECT AVG(H) as avg_height FROM gmass'}`


The following error occuerd: no such column: H
Invoking: `describe_tables` with `{'table_names': 'gmass'}`



Invoking: `run_sqlite_query` with `{'query': 'SELECT AVG(gmass) as avg_weight FROM gmass'}`


The following error occuerd: no such column: gmass
Invoking: `data_dictionary_qa` with `{'question': "What is the meaning of the 'y' table in the database?"}`


["The 'y' table in the database contains data about peoples' height."]


> Finished chain.


{'input': '\nWhat is the average ratio of height to weight of the people in the database?\n',
 'output': ["The 'y' table in the database contains data about peoples' height."]}

----
take 1

```
Invoking: `run_sqlite_query` with `{'query': 'SELECT AVG(height / weight) AS average_ratio FROM people'}`
The following error occuerd: no such table: people
Invoking: `describe_tables` with `{'table_names': 'y'}`
CREATE TABLE y (
        id INTEGER PRIMARY KEY,
        N TEXT,
        H REAL
    )
Invoking: `run_sqlite_query` with `{'query': 'SELECT AVG(H) AS average_ratio FROM y'}`


[(201.89999999999998,)]The average ratio of height to weight of the people in the database is approximately 201.9.
```

----

So, it made a mistake just calculated the average value of H.

--- 
take 2
```text
Invoking: `run_sqlite_query` with `{'query': 'SELECT AVG(height / weight) AS average_ratio FROM gmass'}`
The following error occuerd: no such column: height
Invoking: `data_dictionary_qa` with `{'question': "What is the meaning of the 'gmass' table?"}`
The 'gmass' table contains data about peoples' weight.
Invoking: `data_dictionary_qa` with `{'question': "What is the meaning of the 'y' table?"}`
The 'y' table contains data about people's height.

Invoking: `run_sqlite_query` with `{'query': 'SELECT AVG(y.height / gmass.weight) AS average_ratio FROM y INNER JOIN gmass ON y.id = gmass.id'}`

The following error occuerd: no such column: y.height
Invoking: `run_sqlite_query` with `{'query': 'SELECT AVG(gmass.height / gmass.weight) AS average_ratio FROM gmass'}`


The following error occuerd: no such column: gmass.height
Invoking: `describe_tables` with `{'table_names': ['gmass']}`


CREATE TABLE gmass (
        id INTEGER PRIMARY KEY,
        N TEXT,
        W REAL
    )
Invoking: `data_dictionary_qa` with `{'question': "What is the meaning of the 'gmass' table columns?"}`


The 'gmass' table has a column named 'W' which represents the weight of a person in pounds.
Invoking: `describe_tables` with `{'table_names': ['y']}`


CREATE TABLE y (
        id INTEGER PRIMARY KEY,
        N TEXT,
        H REAL
    )
Invoking: `data_dictionary_qa` with `{'question': "What is the meaning of the 'y' table columns?"}`


The meaning of the columns in the 'y' table is not provided in the given context.
Invoking: `run_sqlite_query` with `{'query': 'SELECT AVG(y.H / gmass.W) AS average_ratio FROM y INNER JOIN gmass ON y.id = gmass.id'}`


[(1.3564722162000782,)]The average ratio of height to weight of the people in the database is approximately 1.356.
```

---

take 3


```text
Invoking: `data_dictionary_qa` with `average ratio of height to weight`


To calculate the average ratio of height to weight, you would need to divide the total height by the total weight. However, since I don't have access to the specific data in your tables, I cannot provide you with the exact average ratio.
Invoking: `describe_tables` with `y`


CREATE TABLE y (
        id INTEGER PRIMARY KEY,
        N TEXT,
        H REAL
    )
Invoking: `describe_tables` with `gmass`



Invoking: `describe_tables` with `gmass`



Invoking: `describe_tables` with `gmass`



Invoking: `run_sqlite_query` with `SELECT AVG(y.H / gmass.W) AS average_ratio FROM y JOIN gmass ON y.id = gmass.id`


[(1.3564722162000782,)]The average ratio of height to weight of the people in the database is approximately 1.36.
```